In [62]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-experimental
# !pip install PyPDF2
# !pip install langchain_groq

In [53]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from rich import print
import PyPDF2
import requests
from bs4 import BeautifulSoup
from google.colab import userdata

In [54]:
# Embedding Model
embedding_model = HuggingFaceInferenceAPIEmbeddings(api_key =userdata.get('HF_TOKEN'),model_name="BAAI/bge-base-en-v1.5")

# Defining the text for reference

In [11]:
def get_content(url:str):

  # Send a GET request to the webpage
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the HTML content using BeautifulSoup
      soup = BeautifulSoup(response.text, "html.parser")

      # Extract the main content (text)
      # This site has the main text within <font> tags
      content = soup.find_all("font")

      # Combine and clean the extracted text
      full_text = "\n".join([element.get_text() for element in content])

      return full_text
  else:
      print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

url = "https://paulgraham.com/persistence.html"
# essay = get_content(url)
# print(essay)

The above code is just for your reference that if you want to explore the online blogs then you can simply pass the url and it will be automatically extracted.

In [31]:
# Open the PDF file in binary mode
with open("/content/RAG.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    content = ""

    # Calculate the number of pages to read (excluding the last 3 pages)
    total_pages = len(reader.pages)
    pages_to_read = total_pages - 5

    # Extract text from the pages, excluding the last 3
    for page_num in range(pages_to_read):
        content += reader.pages[page_num].extract_text()

## Technqiue 1 : Character Text Splitter

In [43]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap=200, separator='', strip_whitespace=True)
docs_technique1 = text_splitter.create_documents([content])

In [48]:
print("Total documents created:",len(docs_technique1))
print(docs_technique1[1])

Total documents created: 92

Document(
    page_content='particularly for knowledge-intensive tasks, and allows\nfor continuous knowledge updates and 
integration of domain-\nspecific information. RAG synergistically merges LLMs’ intrin-\nsic knowledge with the 
vast, dynamic repositories of external\ndatabases. This comprehensive review paper offers a detailed\nexamination 
of the progression of RAG paradigms, encompassing\nthe Naive RAG, the Advanced RAG, and the Modular RAG.\nIt 
meticulously scrutinizes the tripartite foundation of RAG\nframeworks, which includes the retrieval, the generation
and the\naugmentation techniques. The paper highlights the state-of-the-\nart technologies embedded in each of 
these critical components,\nproviding a profound understanding of the advancements in RAG\nsystems. Furthermore, 
this paper introduces up-to-date evalua-\ntion framework and benchmark. At the end, this article delineates\nthe 
challenges currently faced and points out prospective avenues\nfor research and development1.\nIndex Terms —Large 
language model, retri'
)

## Technique 2 : Recursive text splitter

In [46]:
text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
docs_technique2 = text_splitter_recursive.create_documents([content])

In [49]:
print("Total documents created:",len(docs_technique2))
print(docs_technique2[1])

Total documents created: 93

Document(
    page_content='generation, particularly for knowledge-intensive tasks, and allows\nfor continuous knowledge 
updates and integration of domain-\nspecific information. RAG synergistically merges LLMs’ intrin-\nsic knowledge 
with the vast, dynamic repositories of external\ndatabases. This comprehensive review paper offers a 
detailed\nexamination of the progression of RAG paradigms, encompassing\nthe Naive RAG, the Advanced RAG, and the 
Modular RAG.\nIt meticulously scrutinizes the tripartite foundation of RAG\nframeworks, which includes the 
retrieval, the generation and the\naugmentation techniques. The paper highlights the state-of-the-\nart 
technologies embedded in each of these critical components,\nproviding a profound understanding of the advancements
in RAG\nsystems. Furthermore, this paper introduces up-to-date evalua-\ntion framework and benchmark. At the end, 
this article delineates\nthe challenges currently faced and points out prospective avenues\nfor research and 
development1.'
)

## Semantic Splitting

In [56]:
%%time
text_splitter_semantic = SemanticChunker(embedding_model)
docs_technique3 = text_splitter_semantic.create_documents([content])

CPU times: user 364 ms, sys: 73.5 ms, total: 438 ms
Wall time: 1.43 s


In [57]:
print("Total documents created:",len(docs_technique3))
print(docs_technique3[1])

Total documents created: 26

Document(
    page_content='At the end, this article delineates\nthe challenges currently faced and points out prospective 
avenues\nfor research and development1. Index Terms —Large language model, retrieval-augmented gen-\neration, 
natural language processing, information retrieval\nI. I NTRODUCTION\nLARGE language models (LLMs) have achieved 
remark-\nable success, though they still face significant limitations,\nespecially in domain-specific or 
knowledge-intensive tasks [1],\nnotably producing “hallucinations” [2] when handling queries\nbeyond their training
data or requiring current information. To\novercome challenges, Retrieval-Augmented Generation (RAG)\nenhances LLMs
by retrieving relevant document chunks from\nexternal knowledge base through semantic similarity calcu-\nlation. By
referencing external knowledge, RAG effectively\nreduces the problem of generating factually incorrect content. Its
integration into LLMs has resulted in widespread adoption,\nestablishing RAG as a key technology in advancing 
chatbots\nand enhancing the suitability of LLMs for real-world applica-\ntions. RAG technology has rapidly 
developed in recent years, and\nthe technology tree summarizing related research is shown\nCorresponding 
Author.Email:haofen.wang@tongji.edu.cn\n1Resources are available at https://github.com/Tongji-KGLLM/\nRAG-Surveyin 
Figure 1. The development trajectory of RAG in the era\nof large models exhibits several distinct stage 
characteristics. Initially, RAG’s inception coincided with the rise of the\nTransformer architecture, focusing on 
enhancing language\nmodels by incorporating additional knowledge through Pre-\nTraining Models (PTM). This early 
stage was characterized\nby foundational work aimed at refining pre-training techniques\n[3]–[5].The subsequent 
arrival of ChatGPT [6] marked a\npivotal moment, with LLM demonstrating powerful in context\nlearning (ICL) 
capabilities. RAG research shifted towards\nproviding better information for LLMs to answer more com-\nplex and 
knowledge-intensive tasks during the inference stage,\nleading to rapid development in RAG studies. As 
research\nprogressed, the enhancement of RAG was no longer limited\nto the inference stage but began to incorporate
more with LLM\nfine-tuning techniques. The burgeoning field of RAG has experienced swift growth,\nyet it has not 
been accompanied by a systematic synthesis that\ncould clarify its broader trajectory. This survey endeavors 
to\nfill this gap by mapping out the RAG process and charting\nits evolution and anticipated future paths, with a 
focus on the\nintegration of RAG within LLMs. This paper considers both\ntechnical paradigms and research methods, 
summarizing three\nmain research paradigms from over 100 RAG studies, and\nanalyzing key technologies in the core 
stages of “Retrieval,”\n“Generation,” and “Augmentation.” On the other hand, current\nresearch tends to focus more 
on methods, lacking analysis and\nsummarization of how to evaluate RAG. This paper compre-\nhensively reviews the 
downstream tasks, datasets, benchmarks,\nand evaluation methods applicable to RAG. Overall, this\npaper sets out to
meticulously compile and categorize the\nfoundational technical concepts, historical progression, and\nthe spectrum
of RAG methodologies and applications that\nhave emerged post-LLMs. It is designed to equip readers 
and\nprofessionals with a detailed and structured understanding of\nboth large models and RAG. It aims to 
illuminate the evolution\nof retrieval augmentation techniques, assess the strengths and\nweaknesses of various 
approaches in their respective contexts,\nand speculate on upcoming trends and innovations. Our contributions are 
as follows:\n•In this survey, we present a thorough and systematic\nreview of the state-of-the-art RAG methods, 
delineating\nits evolution through paradigms including naive RAG,arXiv:2312.10997v5  [cs.CL]  27 Mar 20242\nFig. 1.
Technology tree of RAG research. The stages of involv

## Agentic Spliting

In [64]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub
from langchain_groq import ChatGroq
import os

In [65]:
# Loading the LLM model
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
model = ChatGroq(model="llama3-8b-8192")

In [60]:
# Pulling out propositions is done via a well-crafted prompt. Let's pull it from LangHub, LangChain's home for prompts.
obj = hub.pull("wfh/proposal-indexing",api_key = userdata.get('LANGSMITH'))

The output from a runnable is a json-esque structure in a string. We need to pull the sentences out. I found that LangChain's example extraction was giving me a hard time so I'm doing it manually with a pydantic data class. There is definitely room to improve this.

Create your class then put it in an extraction chain.

In [66]:
# use it in a runnable
runnable = obj | model

# Pydantic data class
class Sentences(BaseModel):
    sentences: List[str]

# Extraction
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=model)

In [67]:
def get_propositions(text):
    runnable_output = runnable.invoke({
    	"input": text
    }).content

    propositions = extraction_chain.run(runnable_output)[0].sentences
    return propositions

In [70]:
test_content = docs_technique2[2].page_content

In [72]:
results = get_propositions(test_content)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'Here is the extracted information in JSON format:\n\n<tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "information_extraction",\n        "parameters": {\n          "info": {\n            "sentences": [\n              "LARGE language models have achieved remarkable success, though they still face significant limitations.",\n              "LARGE language models face significant limitations, especially in domain-specific or knowledge-intensive tasks.",\n              "Domain-specific or knowledge-intensive tasks are challenging for LARGE language models.",\n              "LARGE language models produce \'hallucinations\' when handling queries beyond their training data or requi